# Линейная регрессия 

### Imports

In [ ]:
import random
import numpy as np
import torch
from torch.utils import data
from torch import nn
import matplotlib.pyplot as plt

In [ ]:
SEED = 19
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)

Сгенерируем свой "игрушечный" набор данных и решим задачу регрессии.

In [ ]:
def synthetic_data(w, b, num_examples):
  X = np.random.normal(0, 1, size=(num_examples, len(w)))
  y = np.dot(X, w) + b
  y += np.random.normal(0, 0.01, size=y.shape)
  return X, y.reshape((-1,1))

Зададим фактические значения w и b, которые и будем находить с использованием различных ML методов. Зная фактические w и b, можем найти фактические значения y :)

In [ ]:
true_w = np.array([2, -3.4])
true_b = 4.2
X, y = synthetic_data(true_w, true_b, 1000)

In [ ]:
X

In [ ]:
y[:6]

In [ ]:
plt.scatter(X[:,0], y)

In [ ]:
plt.scatter(X[:,1], y)

## Аналитическое решение 

### $$  \quad \hat{y_i} = w_1x_{1i} + w_2x_{2i}+b $$
### $$ W =(X^TX)^{-1} X^Ty $$
### $$W - параметры, \: X - обучающая \: выборка, \: y - таргет$$

Если хотим этой формулой найти еще и b, то надо добавидь нувую фичу, которая равна 1 для каждого объекта
(X = np.concatenate((np.ones((len(X), 1)), X), axis=1))

In [ ]:
class MultipleLinearRegression: 
    def fit(self, X, y):
        self.coeffs = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        
    def predict(self, X):
        
        result = X.dot(self.coeffs)
        
        # или
        # result = np.zeros(len(X))
        # for i in range(X.shape[1]):
        #     result += X[:, i] * self.coeffs[i]

        
        return result
    
    def coeffs(self):
        return self.coeffs

In [ ]:
mlp = MultipleLinearRegression()

In [ ]:
mlp.fit(X, y)

In [ ]:
y_pred = mlp.predict(X)

In [ ]:
y_pred[:6]

In [ ]:
mlp.coeffs

## Градиентный спуск с нуля

### $$ MSE = \frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y_i})^2 \quad \textrm{где} \quad \hat{y_i} = w_1x_{1i} + w_2x_{2i}+b $$
### $$𝑓(w_1,w_2,𝑏)= \frac{1}{n}\sum_{i=1}^{n}(y_i - (w_1x_{1i} + w_2x_{2i}+b))^2$$
### $$ [f(g(x))]' = f'(g(x)) * g(x)' \: - \textrm{chain rule}$$
### $$\frac{\partial f}{\partial w_1} = \frac{1}{n}\sum_{i=1}^{n}-2x_{1i}(y_i - (w_1x_{1i} + w_2x_{2i}+b))$$
### $$\frac{\partial f}{\partial b} = \frac{1}{n}\sum_{i=1}^{n}-2(y_i - (w_1x_{1i} + w_2x_{2i}+b))$$

### С использованием numpy

In [ ]:
# Способ реализации бустинга
def gradient_descent(X, y, lr=0.05, num_epoch=100):
    w = np.random.normal(0, 0.01, (2,1))
    b = np.random.normal(1)
    N = len(X)
    for _ in range(num_epoch):    
        # Поманипулируйте с формулами и поймите как пепеходим к матричным операциям
        f = y - (X.dot(w) + b)
        w = w - (lr * (-2 * X *f ).sum(axis=0)).reshape(2,1) / N 
        b -= lr * (-2 * f.sum() / N)      
    
    return w, b

In [ ]:
# Способ реализации бустинга в классе (так красиво и удобнее использовать)
class MultipleLinearRegression:
    def fit(self, X, y, lr=0.05, num_epoch=100):
        self.w = np.random.normal(0, 0.01, (2,1))
        self.b = np.random.normal(1)
        N = len(X)
        for _ in range(num_epoch):    
            f = y - (X.dot(self.w) + self.b)
            self.w = self.w - (lr * (-2 * X *f ).sum(axis=0)).reshape(2,1) / N
            self.b = self.b - lr * (-2 * f.sum() / N)  
            
    def predict(self, X):
        return X.dot(self.w) + self.b
            
    def coeffs(self):
        return self.w, self.b

In [ ]:
mlp = MultipleLinearRegression()

In [ ]:
mlp.fit(X, y, lr=0.01, num_epoch=1000)

In [ ]:
y_pred = mlp.predict(X)

In [ ]:
mlp.coeffs()

## Стахостический градиентый спуск

### С использованием numpy

In [ ]:
def SGD(X, y, lr=0.05, num_epoch=10, batch_size=5):
    w = np.random.normal(0, 0.01, (2,1))
    b = np.random.normal(1)
    
    for _ in range(num_epoch):
        indexes = np.random.randint(0, len(X), batch_size)
        Xs = np.take(X, indexes, axis=0)
        ys = np.take(y, indexes, axis=0)
        Ns = batch_size
    
        f = ys - (Xs.dot(w) + b)
        w = w - (lr * (-2 * Xs *f ).sum(axis=0)).reshape(2,1) / Ns 
        b -= lr * (-2 * f.sum() / Ns)           
    
    return w, b

In [ ]:
# В классе попробуйте реализовать самостоятельно 
class MultipleLinearRegression:
    def fit():
        # твой код здесь
        pass
        
    def predict():
        # твой код здесь
        pass
            
    def coeffs():
        # твой код здесь
        pass

### С использованием  Pytorch autograd

In [ ]:
def synthetic_data(w, b, num_examples):
  X = torch.normal(0, 1, (num_examples, len(w)))
  y = torch.matmul(X, w) + b
  y += torch.normal(0, 0.01, y.shape)
  return X, y.reshape((-1,1))

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
X, y = synthetic_data(true_w, true_b, 1000)

In [ ]:
X

In [ ]:
y[:5]

In [ ]:
plt.scatter(X[:,1].numpy(), y.numpy())

In [ ]:
def data_iter(batch_size, features, labels):
  num_examples = len(features)
  indices = list(range(num_examples))
  random.shuffle(indices)
  for i in range(0, num_examples, batch_size):
      batch_indices = torch.tensor(
          indices[i: min(i + batch_size, num_examples)])
      yield features[batch_indices], labels[batch_indices]


In [ ]:
w = torch.normal(0, 0.01, (2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [ ]:
def linreg(X, w, b):
  return torch.matmul(X, w) + b

In [ ]:
def squared_loss(y_hat, y):
  return (y_hat - y.reshape(y_hat.shape))**2/2

In [ ]:
def sgd(params, lr, batch_size):
  with torch.no_grad():
    for param in params:
      param -= lr * param.grad / batch_size
      param.grad.zero_()

In [ ]:
lr = 0.005
num_epochs = 100
net = linreg
loss = squared_loss
optimizer = sgd 
batch_size = 10

In [ ]:
batch_size

In [ ]:
for epoch in range(num_epochs):
  for X_train, y_ytain in data_iter(batch_size, X, y):
    l = loss(net(X_train, w, b), y_ytain)
    l.sum().backward()
    optimizer([w, b], lr, batch_size)

  with torch.no_grad():
    train_l = loss(net(X_train, w, b), y_ytain)
    print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

In [ ]:
w

In [ ]:
b

### С использованием Pytorch API

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [ ]:
def load_array(data_arrays, batch_size, is_train=True):
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size=batch_size, shuffle=is_train)

In [ ]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [ ]:
net = nn.Sequential(nn.Linear(2,1))

In [ ]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

In [ ]:
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [ ]:
num_epoch = 3
for epoch in range(num_epochs):
  for X, y in data_iter:
    y_pred = net(X)
    l = loss(y_pred, y)
    l.backward()
    trainer.step()
    trainer.zero_grad()

  l = loss(net(features), labels)
  print(f'epoch {epoch + 1}, loss {l:f}')

In [ ]:
w

In [ ]:
b